## 02. Получаем данные из видео

В качестве дополнительного исходного материала попробуем использовать ролики с YouTube. Для того, чтобы с ними работать, установим библиотеку [`PyTube`](https://pytube.io/).

In [1]:
%pip install pytube

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


Возможно, прежде чем выполнять следующий код, вам потребуется перезапустить Kernel.

### Скачиваем звуковые дорожки к видео

Посмотрим, как можно обратиться к ролику на YouTube по ссылке:

In [1]:
from pytube import YouTube

yt = YouTube("https://www.youtube.com/watch?v=2Ggj9AeiqO8")
yt.title, yt.streams

('Гарри Поттер – главная книга 30-летних | ФАЙБ',
 [<Stream: itag="18" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="313" mime_type="video/webm" res="2160p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="271" mime_type="video/webm" res="1440p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="25fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="247" mime_type="video/webm" res="720p" fps="25fps" vcodec="vp9" progr

Мы видим, что с каждым видео связано несколько потоков, включая звуковые. Мы можем отфильтровать нужные нам потоки, взять первый из них, и сохранить на диск:

In [2]:
yt.streams.filter(
    mime_type="audio/webm"
).first()  # .download(output_path="../audio",filename="1.opus")

<Stream: itag="249" mime_type="audio/webm" abr="50kbps" acodec="opus" progressive="False" type="audio">

Возьмём теперь коллекцию видео, и сохраним все звуковые дорожки от них:

In [4]:
videos = [
    "https://www.youtube.com/watch?v=2Ggj9AeiqO8",
    "https://www.youtube.com/watch?v=AZz-PRuTsjE",
    "https://www.youtube.com/watch?v=omsSXJECMtA",
]

for i, url in enumerate(videos):
    yt = YouTube(url)
    print(f"Downloading {yt.title}")
    yt.streams.filter(mime_type="audio/webm").first().download(
        output_path="content/audio", filename=f"{i}.opus"
    )

Для того, чтобы SpeechKit мог распознать речь в файле, файл должен быть в определённом формате и битрейте. Поэтому откроем все наши звуковые дорожки, и сделаем им ресамплинг к частоте 8 kHz с помощью библиотеки `librosa`. Этот процесс может занять некоторое время.

> Вы также можете произвести преобразование форматов и битрейта из командной строки с помощью утилиты `ffmpeg`

In [5]:
import glob

import librosa
import soundfile as sf

target_sr = 8000
for fn in glob.glob("content/audio/*.opus"):
    print(f"Processing {fn}")
    au, sr = librosa.load(fn, sr=target_sr)
    sf.write(fn.replace(".opus", ".ogg"), au, target_sr, format="ogg", subtype="opus")

Processing content/audio/0.opus


<ipython-input-5-0ee5600144a0>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processing content/audio/1.opus


<ipython-input-5-0ee5600144a0>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processing content/audio/2.opus


<ipython-input-5-0ee5600144a0>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


### Распознаём речь с помощью SpeechKit

У SpeechKit есть несколько режимов работы. Для распознавания больших файлов лучше всего подходит [асинхронное распознавание](https://cloud.yandex.ru/docs/speechkit/stt/transcribation). Для асинхронного распознавания необходимо, чтобы исходные файлы лежали в хранилище Yandex S3.

Копируем аудио-файлы в хранилище S3, подключенное к DataSphere, чтобы можно было использовать их для распознавания речи.

> **ВАЖНО:** В рамках мастер-класса сразу несколько участников могут работать с одним хранилищем s3. Поэтому используйте уникальные имена директорий для своих файлов. В данном примере я использую имя **shwars**, которое прошу Вас изменить на какой-то уникальный идентификатор!

In [18]:
user = "shwars"

import os

os.environ["user"] = user

In [20]:
!mkdir -p /home/jupyter/datasphere/s3/s3store/$user/audio
!cp content/audio/*.ogg /home/jupyter/datasphere/s3/s3store/$user/audio

Для работы со Speech Kit потребуется [создать ключ API](https://cloud.yandex.ru/docs/iam/operations/api-key/create). Далее, чтобы не указывать ключ в программном коде, нужно определить ключи как **секреты** в DataSphere. После того, как вы это сделаете, ключи станут доступны просто как переменные окружения.

Весь процесс работы [описан в документации](https://cloud.yandex.ru/docs/speechkit/stt/api/transcribation-api).

In [21]:
import os

api_key = os.environ["api_key"]

Для запуска распознавания конкретного файла необходимо сделать соответствующий POST-запрос.

In [22]:
import requests


def submit_for_sr(audio_file):
    j = {
        "config": {
            "specification": {
                "languageCode": "ru-RU",
            }
        },
        "audio": {"uri": audio_file},
    }
    res = requests.post(
        "https://transcribe.api.cloud.yandex.net/speech/stt/v2/longRunningRecognize",
        json=j,
        headers={"Authorization": f"Api-Key {api_key}"},
    )
    return res.json()["id"]

Инициируем распознавание первого файла. Если всё хорошо, в ответ мы получим `id`, который можно использовать для проверки результата операции.

In [23]:
id = submit_for_sr(f"https://storage.yandexcloud.net/s3store/{user}/audio/1.ogg")
id

'e032oitjigura6bf0140'

Следующую ячейку можно запускать несколько раз, пока вы не получите результат распознавания.

In [24]:
requests.get(
    f"https://operation.api.cloud.yandex.net/operations/{id}",
    headers={"Authorization": f"Api-Key {api_key}"},
).json()

{'done': False,
 'id': 'e032oitjigura6bf0140',
 'createdAt': '2023-11-29T16:53:36Z',
 'createdBy': 'ajegm68gol04oa04moef',
 'modifiedAt': '2023-11-29T16:53:36Z'}

Теперь запустим процесс распознавания для всех наших файлов, и затем будем ждать, пока все результаты не будут получены:

In [25]:
d = {}
for fn in glob.glob(f"/home/jupyter/datasphere/s3/s3store/{user}/audio/*.ogg"):
    ext_name = fn.replace(
        "/home/jupyter/datasphere/s3/", "https://storage.yandexcloud.net/"
    )
    id = submit_for_sr(ext_name)
    print(f"Submitted {fn} -> {id}")
    d[id] = fn

Submitted /home/jupyter/datasphere/s3/s3store/shwars/audio/0.ogg -> e03qpf4hvbfoo049r4es
Submitted /home/jupyter/datasphere/s3/s3store/shwars/audio/1.ogg -> e036ps4snngl3eh4pb1c
Submitted /home/jupyter/datasphere/s3/s3store/shwars/audio/2.ogg -> e033r74s922vgdvk3819


Следующий фрагмент кода будет проверять готовность всех фрагментов распознавания:

In [26]:
import time


def check_ready(id):
    res = requests.get(
        f"https://operation.api.cloud.yandex.net/operations/{id}",
        headers={"Authorization": f"Api-Key {api_key}"},
    )
    res = res.json()
    if res["done"]:
        return res["response"]
    else:
        return None


txt = {}
while True:
    for k, v in d.items():
        if v in txt.keys():
            continue
        res = check_ready(k)
        if res is None:
            print(f"{k} -> waiting")
        else:
            print(f"{k} -> ready")
            txt[v] = " ".join([x["alternatives"][0]["text"] for x in res["chunks"]])
    if len(txt.keys()) == len(d.keys()):
        break
    time.sleep(20)

e03qpf4hvbfoo049r4es -> waiting
e036ps4snngl3eh4pb1c -> ready
e033r74s922vgdvk3819 -> ready
e03qpf4hvbfoo049r4es -> waiting
e03qpf4hvbfoo049r4es -> waiting
e03qpf4hvbfoo049r4es -> waiting
e03qpf4hvbfoo049r4es -> waiting
e03qpf4hvbfoo049r4es -> ready


Теперь сохраним результаты распознавания в текстовые файлы:

In [27]:
os.makedirs(f"/home/jupyter/datasphere/s3/s3store/{user}/text", exist_ok=True)
for k, v in txt.items():
    with open(
        k.replace(".ogg", ".txt").replace("/audio/", "/text/"), "w", encoding="utf-8"
    ) as f:
        f.write(v)

In [29]:
!mkdir -p content/video_text
!cp /home/jupyter/datasphere/s3/s3store/$user/text/* content/video_text

Мы получили дополнительную коллекцию текстовых документов, в которых теперь можно организовать полнотекстовый поиск и построить вопрос-ответного бота. Переходим к следующему этапу работы: `03-LangChainBot.ipynb`